In [1]:
#-*- coding: cp949 -*-
#-*- coding: utf-8 -*- 
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim

from sklearn.datasets import load_iris
import tqdm


iris= load_iris()
data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [2]:
data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [3]:
data_set = data.iloc[:,[0,1,2,3]].values.tolist()
data_set[0]
type(data_set)

list

In [4]:
class CustomDataset(torch.utils.data.Dataset): 
    def __init__(self, df):
    #데이터셋의 전처리를 해주는 부분
    
    #sepal lenght,sepal width, petal length, petal width
    #.iloc 이후 tolist()를 해줌
    #왜나하면 df의 type이 <class 'pandas.core.frame.DataFrame'> 라서 list로 바꿔줘야함
        self.x_data = df.iloc[:,[0,1,2,3]].values.tolist()
        self.y_data = df.iloc[:, [4]].values.tolist()

    def __len__(self):
    #데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분
        return len(self.x_data)

    def __getitem__(self, idx): 
    #데이터셋에서 특정 1개의 샘플을 가져오는 함수
    #list를 tensor로 변환
        x = torch.tensor(self.x_data[idx], dtype = torch.float32)
        y = torch.tensor(self.y_data[idx], dtype = torch.float32)
        return x,y
        

In [5]:
#데이터로더 확인
import torch
import torchvision.transforms as transforms

dataset = CustomDataset(data)

#batch_size 만큼 묶어서 데이터 (batch_size가 커지면 속도가 빨리짐, gpu 사용 증가)
dataLoader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

dataiter = iter(dataLoader)
feature, target = dataiter.next()
print(feature)
print(target)



tensor([[4.9000, 2.5000, 4.5000, 1.7000],
        [4.4000, 3.2000, 1.3000, 0.2000],
        [6.8000, 3.2000, 5.9000, 2.3000],
        [6.0000, 3.0000, 4.8000, 1.8000]])
tensor([[2.],
        [0.],
        [2.],
        [2.]])


In [6]:
#test, train 비율
split_ratio = 0.7
split_index = int(len(data) * split_ratio)

#train, test셋 분류
#데이터가 불균형하게 분포되어 있을 경우 사용 x
train_dataset = CustomDataset(data[:split_index])
test_dataset = CustomDataset(data[split_index:])

#데이터 로더
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

print(len(train_dataloader))
print(len(test_dataloader))

27
12


In [7]:
"""
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
"""

class iris_model(nn.Module):
    def __init__(self, input_dim):
        super(iris_model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
        #target 값이 0~2까지 있음
        #그래서 마지막 layer의 output이 3
        self.layer3 = nn.Linear(50, 3)
        
    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.softmax(self.layer3(x), dim=1)
        return x

#input_dim 입력받아야 함
#input_dim은 feature의 개수
learning_rate = 0.1
model     = iris_model(input_dim=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn   = nn.CrossEntropyLoss()
model = iris_model(input_dim = 4)
#model=Net(input_dim,hidden_dim,output_dim)
#def __init__(self,D_in,H,D_out):



In [8]:
print(len(train_dataloader))
print(len(test_dataloader))

27
12


In [9]:
n_epochs=1000
loss_list=[]
cnt = 0
#n_epochs


#train = 27
#test = 12
for epoch in range(n_epochs):
    running_loss = 0.0

    for x, y in train_dataloader:
      #clear gradient 
        pred = model(x)


        optimizer.zero_grad()
        loss=loss_fn(pred, y.squeeze().long().view(-1))
        loss.backward()
        optimizer.step()
        
        loss_list.append(loss.data)
        running_loss += loss.item()
        #print(loss.item())
    cost = running_loss / len(train_dataloader)
    
    if(epoch % 100 == 0):
        print("epoch %d, loss : %.3f" %(epoch, cost))


epoch 0, loss : 1.145
epoch 100, loss : 1.138
epoch 200, loss : 1.145
epoch 300, loss : 1.143
epoch 400, loss : 1.145
epoch 500, loss : 1.145
epoch 600, loss : 1.143
epoch 700, loss : 1.143
epoch 800, loss : 1.143
epoch 900, loss : 1.145
